# Relevant Segment Extraction (RSE) for Enhanced RAG

이 노트북에서는 RAG 시스템의 컨텍스트 품질을 향상시키기 위해 Relevant Segment Extraction(RSE) 기법을 구현합니다. 단순히 분리된 청크들을 검색하는 대신, 언어 모델에 더 나은 컨텍스트를 제공하는 연속적인 텍스트 세그먼트를 식별하고 재구성합니다.

## Key Concept

관련 청크들은 문서 내에서 함께 클러스터링되는 경향이 있습니다. 이러한 클러스터를 식별하고 연속성을 유지함으로써 LLM이 작업할 수 있는 더 일관된 컨텍스트를 제공합니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [1]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import re

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

## PDF 파일에서 텍스트 추출하기

In [2]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

In [3]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

2023 핵심 개정세법
01
2024 달라지는 세금제도
(국민·기업 납세자용)

[유의사항]
'2023 핵심 개정세법'은 국회에서 의결된 세법 개정사항을 모두 포괄하고 있으나, 시행령·
시행규칙의 경우 2023.7월 발표한 ‘2023 세법개정안'을 중심으로 개정세법과 관련된 내용의
경우 그대로 반영하였습니다. 그러므로 정부의 시행령, 시행규칙 개정 과정에서 일부 변동될
수 있고 새로이 추가 제정될 수도 있습니다. 아울러 실무상 적용할 때는 반드시 개정세법의
구체적인 조문을 확인하셔야 합니다.

국민·기업 납세자용
2024 달라지는 세금제도
2024 달라지는 세금제도
2023 세목별 핵심 개정세법
2023 개정세법 현행-개정사항 비교
01
부동산 세금제도
●(조특법) 연 단위 양도세 감면한도 악용방지를 위해 감면한도 산정방법 조정
양도소득세 산정 및 감면이 연단위로 이뤄지는 점을 감안하여 ▲토지의 일부를
양도한 날부터 소급하여 1년 내 토지를 분할한 경우 분필한 토지 또는 토지 지분



## 추출된 텍스트 chunking
pdf 에서 추출된 텍스트를 얻은 후, 검색 정확성을 향상시키기 위해 이를 더 작고 겹치는 청크로 나눕니다.

In [4]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap) 단계로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트를 청크 리스트에 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  

# OpenAI API 클라이언트 설정하기

In [5]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [27]:
import numpy as np

class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, top_k=5):
        """
        시맨틱 서치 수행

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 포함된 상위 k개 유사 항목.
        """
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(top_k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## 임베딩 생성

In [28]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

Using device: cuda


## RSE를 사용한 문서 처리
이제 핵심 RSE 기능을 구현해 보겠습니다.

In [29]:
def process_document(file_path, chunk_size=800):
    """
    문서를 전처리하여 RSE 시스템에 적합한 형태로 변환합니다.
    
    Args:
        file_path (str): 파일 경로
        chunk_size (int): 각 청크의 크기
        
    Returns:
        Tuple[List[str], SimpleVectorStore, Dict]: chunk, 벡터 저장소, 문서 정보
    """
    print("Extracting text from document...")
    # PDF 파일에서 텍스트 추출
    # text = extract_text_from_pdf(pdf_path)

    # 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    print("Chunking text into non-overlapping segments...")
    # 추출된 텍스트를 청크로 나눕니다.
    chunks = chunk_text(extracted_text, chunk_size, overlap=0)
    print(f"Created {len(chunks)} chunks")
    
    print("Generating embeddings for chunks...")
    # 텍스트 청크의 임베딩 생성
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=1)
    
    # 벡터 저장소 생성
    vector_store = SimpleVectorStore()
    
    # 각 청크와 임베딩을 vector store에 저장
    metadata = [{"chunk_index": i, "source": file_path} for i in range(len(chunks))]
    vector_store.add_item(chunks, chunk_embeddings, metadata)
    
    # 원본 문서 구조 추적
    doc_info = {
        "chunks": chunks,
        "source": file_path,
    }
    
    return chunks, vector_store, doc_info

## RSE 핵심 알고리즘: 청크 값 계산 및 최적 세그먼트 찾기
이제 문서를 처리하고 청크에 대한 임베딩을 생성하는 데 필요한 함수를 준비했으므로, RSE의 핵심 알고리즘을 구현할 수 있습니다.

In [30]:
def calculate_chunk_values(query, chunks, vector_store, irrelevant_chunk_penalty=0.2):
    """
    관련성과 위치를 결합하여 청크 값을 계산합니다.
    
    Args:
        query (str): 쿼리 텍스트
        chunks (List[str]): 문서 청크 리스트
        vector_store (SimpleVectorStore): 청크가 포함된 벡터 저장소
        irrelevant_chunk_penalty (float): 관련성이 없는 청크에 대한 패널티
        
    Returns:
        List[float]: 청크 값 리스트
    """
    # 쿼리 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
    
    # 유사도 점수가 있는 모든 청크 가져오기
    num_chunks = len(chunks)
    results = vector_store.similarity_search(query_embedding, top_k=num_chunks)
    
    # 청크 인덱스와 관련성 점수 매핑
    relevance_scores = {result["metadata"]["chunk_index"]: result["score"] for result in results}
    
    # 청크 값 계산 (관련성 점수에서 패널티 적용)
    chunk_values = []
    for i in range(num_chunks):
        # 관련성 점수 가져오기 또는 결과에 없는 경우 0으로 설정
        score = relevance_scores.get(i, 0.0)
        # 관련성이 없는 청크에 대한 패널티 적용
        value = score - irrelevant_chunk_penalty
        chunk_values.append(value)
    
    return chunk_values

In [31]:
def find_best_segments(chunk_values, max_segment_length=20, total_max_length=30, min_segment_value=0.2):
    """
    최대 합 하위 배열 알고리즘의 변형을 사용하여 최적의 연속 텍스트 세그먼트를 찾습니다.
    
    Args:
        chunk_values (List[float]): 각 청크의 값
        max_segment_length (int): 단일 세그먼트의 최대 길이
        total_max_length (int): 모든 세그먼트에 대한 최대 총 길이
        min_segment_value (float): 세그먼트로 간주되기 위한 최소 값
        
    Returns:
        List[Tuple[int, int]]: 최적 세그먼트의 (start, end) 인덱스 리스트
    """
    print("Finding optimal continuous text segments...")
    
    best_segments = []
    segment_scores = []
    total_included_chunks = 0
    
    # 세그먼트 찾기
    while total_included_chunks < total_max_length:
        best_score = min_segment_value  # 세그먼트로 간주되기 위한 최소 값  
        best_segment = None
        
        # 가능한 모든 시작 위치 시도
        for start in range(len(chunk_values)):
            # 이미 선택된 세그먼트에 포함된 경우 건너뜁니다.
            if any(start >= s[0] and start < s[1] for s in best_segments):
                continue
                
            # 가능한 모든 세그먼트 길이 시도
            for length in range(1, min(max_segment_length, len(chunk_values) - start) + 1):
                end = start + length
                
                # 이미 선택된 세그먼트에 포함된 경우 건너뜁니다.
                if any(end > s[0] and end <= s[1] for s in best_segments):
                    continue
                
                # 세그먼트 값을 청크 값의 합으로 계산
                segment_value = sum(chunk_values[start:end])
                
                # 이 세그먼트가 더 좋은 경우 업데이트
                if segment_value > best_score:
                    best_score = segment_value
                    best_segment = (start, end)
        
        # 좋은 세그먼트를 찾은 경우 추가
        if best_segment:
            best_segments.append(best_segment)
            segment_scores.append(best_score)
            total_included_chunks += best_segment[1] - best_segment[0]
            print(f"Found segment {best_segment} with score {best_score:.4f}")
        else:
            # 더 좋은 세그먼트를 찾을 수 없음   
            break
    
    # 읽기 쉽도록 세그먼트를 시작 위치에 따라 정렬
    best_segments = sorted(best_segments, key=lambda x: x[0])
    
    return best_segments, segment_scores

## Reconstructing and Using Segments for RAG

In [32]:
def reconstruct_segments(chunks, best_segments):
    """
    청크 인덱스를 기반으로 텍스트 세그먼트를 재구성합니다.
    
    Args:
        chunks (List[str]): 모든 문서 청크 리스트
        best_segments (List[Tuple[int, int]]): 세그먼트의 (start, end) 인덱스 리스트
        
    Returns:
        List[str]: 재구성된 텍스트 세그먼트 리스트
    """
    reconstructed_segments = []  # 재구성된 세그먼트를 저장할 빈 리스트 초기화
    
    for start, end in best_segments:
        # 이 세그먼트의 청크를 결합하여 완전한 세그먼트 텍스트 생성
        segment_text = " ".join(chunks[start:end])
        # 세그먼트 텍스트와 범위를 reconstructed_segments 리스트에 추가
        reconstructed_segments.append({
            "text": segment_text,
            "segment_range": (start, end),
        })
    
    return reconstructed_segments  # 재구성된 텍스트 세그먼트 리스트 반환

In [33]:
def format_segments_for_context(segments):
    """
    세그먼트를 LLM에 대한 컨텍스트 문자열로 포맷합니다.
    
    Args:
        segments (List[Dict]): 세그먼트 딕셔너리 리스트
        
    Returns:
        str: 포맷팅된 컨텍스트 텍스트
    """
    context = []  # 포맷팅된 컨텍스트를 저장할 빈 리스트 초기화
    
    for i, segment in enumerate(segments):
        # 각 세그먼트에 대한 헤더 생성 (인덱스와 청크 범위)
        segment_header = f"SEGMENT {i+1} (Chunks {segment['segment_range'][0]}-{segment['segment_range'][1]-1}):"
        context.append(segment_header)  # 세그먼트 헤더를 컨텍스트 리스트에 추가
        context.append(segment['text'])  # 세그먼트 텍스트를 컨텍스트 리스트에 추가
        context.append("-" * 80)  # 구분 선 추가
    
    # 컨텍스트 리스트의 모든 요소를 더블 뉴라인으로 결합하여 결과 반환
    return "\n\n".join(context)

## Generating Responses with RSE Context

In [34]:
def generate_response(query, context, model_name='gpt-4.1-mini'):

    # AI 어시스턴트의 시스템 프롬프트를 정의합니다.
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

## Complete RSE Pipeline Function

In [35]:
def rag_with_rse(pdf_path, query, chunk_size=800, irrelevant_chunk_penalty=0.2):
    """
    RSE를 사용한 RAG 파이프라인을 구현합니다.
    
    Args:
        pdf_path (str): 문서 경로
        query (str): 사용자 쿼리
        chunk_size (int): 청크 크기
        irrelevant_chunk_penalty (float): 관련성이 없는 청크에 대한 패널티
        
    Returns:
        Dict: 쿼리, 세그먼트, 응답이 포함된 결과
    """
    print("\n=== STARTING RAG WITH RELEVANT SEGMENT EXTRACTION ===")
    print(f"Query: {query}")
    
    # 문서를 처리하여 텍스트 추출, 청크로 나누고 임베딩 생성
    chunks, vector_store, doc_info = process_document(pdf_path, chunk_size)
    
    # 쿼리에 대한 관련성 점수와 청크 값 계산
    print("\nCalculating relevance scores and chunk values...")
    chunk_values = calculate_chunk_values(query, chunks, vector_store, irrelevant_chunk_penalty)
    
    # 청크 값에 따라 텍스트 세그먼트 찾기
    best_segments, scores = find_best_segments(
        chunk_values, 
        max_segment_length=20, 
        total_max_length=30, 
        min_segment_value=0.2
    )
    
    # 최적 청크에서 텍스트 세그먼트 재구성
    print("\nReconstructing text segments from chunks...")
    segments = reconstruct_segments(chunks, best_segments)
    
    # 세그먼트를 언어 모델에 대한 컨텍스트 문자열로 포맷
    context = format_segments_for_context(segments)
    
    # 컨텍스트를 기반으로 응답 생성
    response = generate_response(query, context)
    
    # 결과를 딕셔너리로 컴파일
    result = {
        "query": query,
        "segments": segments,
        "response": response
    }
    
    print("\n=== FINAL RESPONSE ===")
    print(response)
    
    return result

## 표준 검색 방식과의 비교
RSE와 비교하기 위해 표준 검색 방식을 구현해 보겠습니다:

In [36]:
def standard_top_k_retrieval(file_path, query, k=10, chunk_size=800):
    """
    표준 RAG 파이프라인을 구현합니다.
    
    Args:
        file_path (str): 문서 경로
        query (str): 사용자 쿼리
        k (int): 검색할 청크 수
        chunk_size (int): 청크 크기
        
    Returns:
        Dict: 쿼리, 청크, 응답이 포함된 결과
    """
    print("\n=== STARTING STANDARD TOP-K RETRIEVAL ===")
    print(f"Query: {query}")
    
    # 문서를 처리하여 텍스트 추출, 청크로 나누고 임베딩 생성
    chunks, vector_store, doc_info = process_document(file_path, chunk_size)
    
    # 쿼리에 대한 임베딩 생성
    print("Creating query embedding and retrieving chunks...")
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
    
    # 쿼리 임베딩에 따라 상위 k개의 관련성 높은 청크 검색
    results = vector_store.search(query_embedding, top_k=k)
    retrieved_chunks = [result["text"] for result in results]
    
    # 검색된 청크를 컨텍스트 문자열로 포맷
    context = "\n\n".join([
        f"CHUNK {i+1}:\n{chunk}" 
        for i, chunk in enumerate(retrieved_chunks)
    ])
    
    # 컨텍스트를 기반으로 응답 생성
    response = generate_response(query, context)
    
    # 결과를 딕셔너리로 컴파일
    result = {
        "query": query,
        "chunks": retrieved_chunks,
        "response": response
    }
    
    print("\n=== FINAL RESPONSE ===")
    print(response)
    
    return result

## RSE 기법 평가

In [37]:
def evaluate_methods(file_path, query, reference_answer=None):
    """
    RSE와 표준 RAG 비교
    
    Args:
        file_path (str): 문서 경로
        query (str): 사용자 쿼리
        reference_answer (str, optional): 참조 답변
    """
    print("\n========= EVALUATION =========\n")
    
    # RSE 방법 실행
    rse_result = rag_with_rse(file_path, query)
    
    # 표준 RAG 방법 실행
    standard_result = standard_top_k_retrieval(file_path, query)
    
    # 참조 답변이 제공된 경우 응답 평가
    if reference_answer:
        print("\n=== COMPARING RESULTS ===")
        
        # 평가 프롬프트 생성
        evaluation_prompt = f"""
            Query: {query}

            Reference Answer:
            {reference_answer}

            Response from Standard Retrieval:
            {standard_result["response"]}

            Response from Relevant Segment Extraction:
            {rse_result["response"]}

            Compare these two responses against the reference answer. Which one is:
            1. More accurate and comprehensive
            2. Better at addressing the user's query
            3. Less likely to include irrelevant information

            Explain your reasoning for each point.
        """
        
        print("Evaluating responses against reference answer...")
        
        # 지정된 모델을 사용하여 평가 생성
        evaluation = client_openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": "You are an objective evaluator of RAG system responses."},
                {"role": "user", "content": evaluation_prompt}
            ]
        )
        
        # 평가 결과 출력
        print("\n=== EVALUATION RESULTS ===")
        print(evaluation.choices[0].message.content)
    
    # 두 방법의 결과 반환
    return {
        "rse_result": rse_result,
        "standard_result": standard_result
    }

In [38]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# 평가 데이터에서 첫 번째 쿼리를 추출합니다.
query = df['query'][0]

# 평가 데이터에서  정답 답변을 추출합니다.
true_answer = df['generation_gt'][0]

# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# Run evaluation
results = evaluate_methods(file_path, query, true_answer)


========= EVALUATION =========


=== STARTING RAG WITH RELEVANT SEGMENT EXTRACTION ===
Query: 피고의 고지의무 위반으로 인해 원고들이 입은 손해는 무엇으로 정의됩니까?
Extracting text from document...
Chunking text into non-overlapping segments...
Created 35 chunks
Generating embeddings for chunks...


Batches:   0%|          | 0/35 [00:00<?, ?it/s]


Calculating relevance scores and chunk values...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: shapes (1024,) and (35,1024) not aligned: 1024 (dim 0) != 35 (dim 0)